In [57]:
import matplotlib.pyplot as plt
%matplotlib nbagg
import numpy as np

<figure id="figure-3">
<div style="background-color: white">

![](figures/dct.svg)</div>
    
<figcaption style="text-align: center">Figure 3: A DCT can be treated as an N channel filter bank where the coefficients of the
filters are the basis functions.</figcaption></figure>

# 7 The Discrete Cosine Transform (DCT)


The DCT is a method of performing energy compaction that is rather different
from the pyramid method.  It operates on non-overlapping blocks of pixels
(typically $8 \times 8$ pixels in size) by a reversible linear transform
process, such that each block of pixels is replaced by a block of the same
number of transform coefficients.  If all the transform coefficients for a
given block are transmitted unaltered to the decoder, then the original block
of pixels can be exactly recovered by the inverse transform process.

In practise the transform coefficients are quantised before transmission, and
if energy compaction has occurred, then fewer bits will be needed to send the
coefficients than the original pixels.  A key advantage of transform-based
methods is that there is no expansion of the number of samples (the
transformed block is the same size as the original block of pixels), whereas
the previous pyramid method expands the data by
$1 + \frac{1}{4} + \frac{1}{16} + \ldots \approx 1.33$ times, which is not very desirable for data
compression.

## 7.1 Definition of the DCT


The one-dimensional form of the DCT is closely related to the Discrete Fourier
Transform (DFT).  The 1-D $N$-point DCT is defined as follows:


$$
y(k) = \sum_{n=0}^{N-1} C_{kn}\ x(n) \quad \text{for} \quad 0 \le k \le N-1 \\
  \text{where }\quad C_{0n} = \sqrt{\frac{1}{N}}  \\
    \text{and } \quad C_{kn} = \sqrt{\frac{2}{N}}\ \cos
\frac{k(n+\frac{1}{2})\pi}{N} \quad \text{for} \quad 1 \le k \le N-1
$$

The equivalent inverse DCT is:
$$
x(n) = \sum_{k=0}^{N-1} C_{kn}\ y(k) \quad \text{for} \quad 0 \le n \le N-1 \\
 \text{where $C_{kn}$ is defined as above.}\\
$$

(This is actually the Type-II DCT, and the inverse is the Type-III DCT - other types have slightly different relative phases})

We see that the forward transform is equivalent to multiplication of the
$N$-point column vector $[x(0) \ldots x(N-1)]'$ by an $N \times N$ matrix,
containing $C_{kn}$ at each location $(k,n)$, to produce the $N$-point column
vector $[y(0) \ldots y(N-1)]'$.  Similarly the inverse transform is equivalent
to multiplication of the $y$ vector by the transpose of the $C$ matrix to give
the $x$ vector.  In python3 + numpy notation these become:

`y = C @ x` and `x = C.T @ y`

Note that C is an orthonormal matrix since its inverse is just its
transpose (its rows are othogonal to each other and have unit energy).

The two-dimensional version of the DCT (as used for image compression) is a
simple extension of the above 1-D DCT.  For an $N \times N$ block of pixels,
the $N$-point 1-D DCT is first applied to each column of the block to give $N$
columns of coefficients.  Then the same 1-D DCT is applied to the rows of
these coefficients to give the 2-D transform coefficients.

In python3 + numpy notation, if the input block of pixels is matrix X, the output
block of 2-D transformed coefficients Y is given by:

`Y = (C @ (C @ X).T).T` or more simply `Y = C @ X @ C.T`

where C is the 1-D transform matrix as above.  Note that in the 2-D
transform, it does not matter whether the rows or the columns are transformed
first (because the transform is linear and separable).


## 7.2 Applying the DCT to images

Conceptually the 2-D DCT is applied to all non-overlapping $N \times N$ blocks
of pixels in an image (we assume that the image dimensions are exact multiples
of $N$).  However it is simplest and most efficient to perform 1-D
$N$-point DCTs on all the columns of the image first, and then repeat the
operation on the transpose of the result to transform the rows.

**First generate an 8-point 1-D Type-II DCT matrix C8**

In [58]:
from cued_sf2_lab.dct import dct_ii

C8 = dct_ii(8)

print(C8)

[[ 0.35355339  0.35355339  0.35355339  0.35355339  0.35355339  0.35355339
   0.35355339  0.35355339]
 [ 0.49039264  0.41573481  0.27778512  0.09754516 -0.09754516 -0.27778512
  -0.41573481 -0.49039264]
 [ 0.46193977  0.19134172 -0.19134172 -0.46193977 -0.46193977 -0.19134172
   0.19134172  0.46193977]
 [ 0.41573481 -0.09754516 -0.49039264 -0.27778512  0.27778512  0.49039264
   0.09754516 -0.41573481]
 [ 0.35355339 -0.35355339 -0.35355339  0.35355339  0.35355339 -0.35355339
  -0.35355339  0.35355339]
 [ 0.27778512 -0.49039264  0.09754516  0.41573481 -0.41573481 -0.09754516
   0.49039264 -0.27778512]
 [ 0.19134172 -0.46193977  0.46193977 -0.19134172 -0.19134172  0.46193977
  -0.46193977  0.19134172]
 [ 0.09754516 -0.27778512  0.41573481 -0.49039264  0.49039264 -0.41573481
   0.27778512 -0.09754516]]


Take a look at the function `dct_ii` and list `C8` to check
that it agrees with the definitions for $C_{kn}$ given above:

In [59]:
import inspect
import IPython.display
IPython.display.Code(inspect.getsource(dct_ii), language="python")

def dct_ii(N: int) -> np.ndarray:
    """
    Generate the 1D DCT transform matrix of size N.

    Parameters:
    N (int): Size of DCT matrix required

    Returns:
    C (2D np array): 1D DCT transform matrix

    Uses an orthogonal Type-II DCT.
    Y = C * X tranforms N-vector X into Y.
    """
    C = np.ones((N, N)) / np.sqrt(N)
    theta = (np.arange(N) + 0.5) * (np.pi/N)
    g = np.sqrt(2/N)
    for i in range(1, N):
        C[i, :] = g * np.cos(theta*i)

    return C

**Plot the rows of `C8` using `plot(C8.T)`.**

In [60]:
for i in range(8):
    fig, ax = plt.subplots()
    ax.plot(C8.T[i])
fig, ax = plt.subplots()
ax.plot(C8.T[0])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


When we calculate the 1-D transform of an 8-point block of data, each
transform coefficient represents the component of the data that is
correlated with the corresponding row of `C8`.  Hence the
first coefficient represents the dc component, the second one
represents the approximate average slope, and so on. The later
coefficients represent progressively higher frequency components
in the data.

The function `colxfm(X, C8)` will perform a 1-D transform on
the columns of image `X` using `C8`. We can
therefore perform a 2-D transform on `X` by using `colxfm` twice, once with transpose operators, as follows:

In [61]:
from cued_sf2_lab.familiarisation import load_mat_img
from cued_sf2_lab.dct import colxfm

X_pre_zero_mean, cmaps_dict = load_mat_img(img='cued_sf2_lab/lighthouse.mat', img_info='X', cmap_info={'map', 'map2'})
X = X_pre_zero_mean - 128.0
print(np.max(X), np.min(X))
Y = colxfm(colxfm(X, C8).T, C8).T

120.0 -128.0


In [62]:
IPython.display.Code(inspect.getsource(colxfm), language="python")

def colxfm(X: np.ndarray, C: np.ndarray) -> np.ndarray:
    """
    Transforms the columns of X using the tranformation in C.

    Parameters:
        X: Image whose columns are to be transformed
        C: N-size 1D DCT coefficients obtained using dct_ii(N)
    Returns:
        Y: Image with transformed columns

    PS: The height of X must be a multiple of the size of C (N).
    """
    N = len(C)
    m, n = X.shape

    # catch mismatch in size of X
    if m % N != 0:
        raise ValueError('colxfm error: height of X not multiple of size of C')

    Y = np.zeros((m, n))
    # transform columns of each horizontal stripe of pixels, N*n
    for i in range(0, m, N):
        Y[i:i+N, :] = C @ X[i:i+N, :]

    return Y

In `Y`, each $8 \times 8$ block of pixels has been replaced by an
equivalent block of transform coefficients.  The coefficient in the top left
corner of each block represents the dc value of the block of pixels;
coefficients along the top row represent increasing horizontal frequency
components, and along the left column represent increasing vertical frequency
components.  Other coefficients represent various combinations of horizontal
and vertical frequencies, in proportion to their horizontal and vertical
distances from the top left corner.

If we try to display `Y` directly as an image, it is rather
confusing because the different frequency components of each block
are all present adjacent to each other.  

In [63]:
from cued_sf2_lab.familiarisation import plot_image
C4 = dct_ii(4)
Y4 = colxfm(colxfm(X, C4).T, C4).T
fig, ax = plt.subplots()
plot_image(Y4, ax=ax);

<IPython.core.display.Javascript object>

A much more meaningful
image is created if we group all the coefficients of a given type
together into a small sub-image, and display the result as an $8
\times 8$ block of sub-images, one for each coefficient type.  The
function `regroup(Y, N)` achieves this regrouping, where $N$ is
the size of the original transform blocks. You need to ensure that X has zero mean (by subtracting 128) before you start transforming it, otherwise the dc coefficient will be purely positive, whereas the
others are symmetrically distributed about zero. Also, an $N
\times N$ 2-D DCT introduces a gain factor of $N$ in order to
preserve constant total energy between the pixel and transform
domains: we need to divide by $N$ *when displaying* to get back to the expected range.

Hence we can display `Y` meaningfully using:

In [64]:
from cued_sf2_lab.dct import regroup

N = 4
fig, ax = plt.subplots()
Yr = regroup(Y4, N)
plot_image(Yr, ax=ax);

<IPython.core.display.Javascript object>

In [65]:
Yg = regroup(Yr, 64)

fig, ax = plt.subplots()
plot_image(Yg, ax=ax)

<IPython.core.display.Javascript object>

In this image, you should see a small replica of the original in the top left
corner (the dc coefficients), and other sub-images showing various edges from
the original, representing progressively higher frequencies as you move
towards the lower right corner.

<div class="alert alert-block alert-danger">

What do you observe about the energies of the sub-images as frequencies
increase?</div>

In [66]:
# Your code here

ordered = regroup(Y, N)/N
energy = np.zeros((8, 8))
for r in range(0, 224, 32):
    for c in range(0, 224, 32):
        energy[r//32][c//32] = np.sum(ordered[r: r + 32, c: c + 32]**2)

fig, ax = plt.subplots()
im = plot_image(energy, ax=ax)
fig.colorbar(im)

<IPython.core.display.Javascript object>

Now check that you can recover the original image from Y
by carrying out the inverse transform using:

In [67]:
Z = colxfm(colxfm(Y.T, C8.T).T, C8.T)

fig, ax = plt.subplots()
plot_image(Z, ax=ax);
np.std(X-Z)

<IPython.core.display.Javascript object>

1.0352065787911857e-13

**Measure the maximum absolute error between X and Z
to confirm this.**

In [68]:
# Your code here

np.max(np.abs(X- Z))

6.110667527536862e-13

The DCT analyses each $8 \times 8$ block of image pixels into a linear
combination of sixty-four $8 \times 8$ basis functions.  The following will generate
an image comprising these basis functions (the `np.nan`s separate the sub-images as matplotlib draws them as transparent, and the `reshape` function converts from a matrix to a row vector):

In [69]:
import numpy as np
# Stack some NaNs
bases = np.concatenate([np.full((8, 1), np.nan), C8, np.full((8, 1), np.nan)], axis=1)
# Reshape
bases_flat = np.reshape(bases, (-1, 1))
print(bases.shape)
print(bases_flat.shape)
fig, ax = plt.subplots()
im = plot_image(255*bases_flat@bases_flat.T, ax=ax)
fig.colorbar(im);

(8, 10)
(80, 1)


<IPython.core.display.Javascript object>

<div class="alert alert-block alert-danger">

Explain how this image relates to the DCT coefficients.</div>

## 7.3 Quantisation and Coding Efficiency

We are now going to look at the effects of quantising the DCT coefficients
fairly coarsely and determine the entropies of the coefficient sub-images.
At this stage we shall quantise all sub-images with the same step-size,
since they all are the same size and have unit energy gain from the
quantiser to the output image (due to the orthonormal transform matrices).

First quantise the transformed image Y using a step size
of 17 to give Yq.  Then regroup Yq to form
sub-images of each coefficient type as before, to give Yr. These sub-images have different probability distributions and we can take advantage of this later in coding them efficiently. Hence we get a better estimate of the number of bits required to code Yq by looking at the entropies of each of the re-grouped sub-images separately.

**Write a function `dctbpp(Yr, N)` to calculate the total number of bits from a re-grouped image Yr, by using `bpp(Ys)` on each sub-image Ys of Yr, then multiplying each result by the number of pixels in the sub-image, and summing to give the total number of bits.**

<div class="alert alert-block alert-danger">

Visualise Yr and comment on the distributions in each of the sub-images. Use the function `dctbpp(Yr, N)` that you have written to calculate the total number of bits, and compare it with just using `bpp(Yr)`, explaining your results.

</div>

In [70]:
from cued_sf2_lab.laplacian_pyramid import bpp

def dctbpp(Yr, N):
    it = Yr.shape[0]//N
    output = 0
    for r in range(0, Yr.shape[0], it):
        for c in range(0, Yr.shape[0], it):
            Ys = Yr[r:r+it, c:c+it]
            output += bpp(Ys)*Ys.size
    return output


In [71]:
# Your code here
from cued_sf2_lab.laplacian_pyramid import quantise, quant1, quant2
N = 8
Y1 = quant1(Y, 17)
Z1 = colxfm(colxfm(Y1.T, C8.T).T, C8.T)
Y2 = quant2(Y, 17)
Z2 = colxfm(colxfm(Y2, C8.T).T, C8.T)
Yq = quantise(Y, 17)
Zq = colxfm(colxfm(Yq.T, C8.T).T, C8.T)


Yr = regroup(Yq, N)/N
fig, ax = plt.subplots()
plot_image(Z1, ax=ax)
fig, ax = plt.subplots()
plot_image(Z2, ax=ax)
fig, ax = plt.subplots()
plot_image(Zq, ax=ax)
print(dctbpp(Yr, N))
print(bpp(Yr)*Yr.size)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

97467.19741586194
109626.49318603268


<div class="alert alert-block alert-danger">

Now reconstruct the output image `Z` from `Yq` and measure the rms
error (standard deviation) between `X` and `Z`.  Compare this with the
error produced by quantising `X` with a step-size of 17 to give `Xq`.

</div>

In [72]:
# Your code here

Zq = colxfm(colxfm(Yq.T, C8.T).T, C8.T)

fig, ax = plt.subplots()
plot_image(Zq, ax=ax)

print(np.std(X-Zq))
print(np.std(X - quantise(X, 17)))

<IPython.core.display.Javascript object>

3.756757368843634
4.861168497356846


***As with the Laplacian Pyramid, we really need to contrast compression ratios and visual results on compressed images with the same rms error. Re-use your step optimisation code to calculate the (non-integer) step size required in this case for the same rms error as quantising X with a step-size of 17.***

In [73]:
# Your code here

def optimisation_DCT(target_MSE, Y, C, start, stop, tot):
    error = float("inf")
    output = None
    step_sizes = np.linspace(start, stop, tot)
    for step in step_sizes:
        Yq = quantise(Y, step, None)
        Zq = colxfm(colxfm(Yq.T, C.T).T, C.T)
        diff = np.abs(np.std(X-Zq) - target_MSE)
        if diff < error:
            error = diff
            output = step
    return output

print(optimisation_DCT(np.std(X- quantise(X, 17)), Y, C8, 1, 40, 200))

23.73366834170854


<div class="alert alert-block alert-danger">

Calculate the compression ratio for this scheme compared to direct quantisation. Use `dctbpp` to calculate the number of bits needed. Contrast the visual appearance of the DCT-compressed image, the directly quantised image, and the original image.

</div>

In [74]:
# Your code here
step8 = optimisation_DCT(np.std(X- quantise(X, 17)), Y, C8, 1, 40, 200)
def compression_ratio_DCT(X, Y, N, ref_step, step_size):
    output = 0
    Yq= quantise(Y, step_size)
    Yr = regroup(Yq, N)/N
    output += dctbpp(Yr, N)
    Xq = quantise(X, ref_step)
    output = bpp(Xq)*Xq.size/output
    return output

print("compression ratio for 8x8: {}".format(compression_ratio_DCT(Z, Y, 8, 17, step8)))

compression ratio for 8x8: 2.946373050593443


In [75]:
Yq_rms = quantise(Y,15.306532663316583, 15.306532663316583)
Zq_rms = colxfm(colxfm(Yq_rms.T, C8.T).T, C8.T)

fig, axs = plt.subplots(1, 3, figsize=(8, 4))
plot_image(X, ax=axs[0])
axs[0].set(title='Original Image')
plot_image(quantise(X, 17), ax=axs[1])
axs[1].set(title='Direct Quantisation')
plot_image(Zq_rms, ax=axs[2])
axs[2].set(title='8x8 DCT compressed')

<IPython.core.display.Javascript object>

[Text(0.5, 1.0, '8x8 DCT compressed')]

In [76]:
fig, ax = plt.subplots()
fig.tight_layout()
plot_image(Zq_rms, ax=ax)

<IPython.core.display.Javascript object>


## 7.4 Alternative transform sizes

So far, we have concentrated on $8 \times 8$ DCTs using C8
as the 1-D transform matrix.  **Now generate 4-point and 16-point
transform matrices, C4 and C16 using `dct_ii`.**


In [77]:
# Your code here

C4 = dct_ii(4)
C16 = dct_ii(16)


<div class="alert alert-block alert-danger">

Repeat the main measurements from the previous section, so as to obtain
estimates of the number of bits and compression ratios for $4 \times 4$ and $16 \times 16$ DCTs when the
rms errors are equivalent to those in your previous tests.  Also assess the
relative subjective quality of the reconstructed images.</div>

In [78]:
# Your code here
Y4 = colxfm(colxfm(X, C4).T, C4).T
step4 = optimisation_DCT(np.std(X - quantise(X, 17)), Y4, C4, 1, 40, 200 )
Y4q = quantise(Y4, step4)
Z4q = colxfm(colxfm(Y4q.T, C4.T).T, C4.T)
print("number of bits in regrouped image: {}".format(dctbpp(regroup(Y4q, 4)/4, 4)))
print("compression ratio for 4x4: {}".format(compression_ratio_DCT(X, Y4, 4, 17, step4)))
print(step4)
fig, ax = plt.subplots()
plot_image(Z4q, ax=ax)
ax.set(title  = "4x4 DCT")
print("MSE: {}".format(np.std(X - Z4q)))

number of bits in regrouped image: 86165.84626643773
compression ratio for 4x4: 2.647441491067228
23.929648241206028


<IPython.core.display.Javascript object>

MSE: 4.865877067875976


In [79]:
# Your code here
Y8 = colxfm(colxfm(X, C8).T, C8).T
step8 = optimisation_DCT(np.std(X - quantise(X, 17)), Y8, C8, 1, 40, 200 )
Y8q = quantise(Y8, step8)
Z8q = colxfm(colxfm(Y8q.T, C8.T).T, C8.T)
print("number of bits in regrouped image: {}".format(dctbpp(regroup(Y8q, 8)/8, 8)))
print("compression ratio for 8x8: {}".format(compression_ratio_DCT(X, Y8, 8, 17, step8)))
print(step8)
fig, ax = plt.subplots()
plot_image(Z8q, ax=ax)
ax.set(title  = "8x8 DCT")

print("MSE: {}".format(np.std(X - Z8q)))

number of bits in regrouped image: 77423.6773828558
compression ratio for 8x8: 2.946373050593443
23.73366834170854


<IPython.core.display.Javascript object>

MSE: 4.867500738323735


In [80]:
Y16 = colxfm(colxfm(X, C16).T, C16).T
step16 = optimisation_DCT(np.std(X - quantise(X, 17)), Y16, C16, 1, 40, 200 )
Y16q = quantise(Y16, step16)
Z16q = colxfm(colxfm(Y16q.T, C16.T).T, C16.T)
print("compression ratio for 16x16: {}".format(compression_ratio_DCT(X, Y16, 16, 17, step16)))
print("number of bits in regrouped image: {}".format(dctbpp(regroup(Y16q, 16)/16, 4)))
print(step16)
fig, ax = plt.subplots()
plot_image(Z16q, ax=ax)
ax.set(title  = "16x16 DCT")
print("MSE: {}".format(np.std(X - Z16q)))

compression ratio for 16x16: 2.883369722681117
number of bits in regrouped image: 83040.53571582095
22.361809045226128


<IPython.core.display.Javascript object>

MSE: 4.865683652640659


In [81]:

fig, axs = plt.subplots(1, 3, figsize=(10, 5))
plot_image(Z4q[106:226, 8:128], ax=axs[0])
axs[0].set(title="4x4 DCT")
plot_image(Z8q[106:226, 8:128], ax=axs[1])
axs[1].set(title="8x8 DCT")
plot_image(Z16q[106:226, 8:128], ax=axs[2])
axs[2].set(title="16x16 DCT")

<IPython.core.display.Javascript object>

[Text(0.5, 1.0, '16x16 DCT')]


This analysis is in fact slightly biased because with larger transform sizes the function `dctbpp(Yr, N)` will use a greater number of smaller sub-images on which to calculate probability distributions. It may be better to use the same N in this function even when the actual transform changes; however whether this is more predictive of actual coding performance depends on what scanning method is used in the coding scheme.

<div class="alert alert-block alert-danger">

What happens in the limit if you use `dctbpp(Yr, 256)` (i.e. the entropy is calculated independently for each pixel)? Why is this the case, and why isn't this a realistic result?</div>

In [82]:
# Your code here
C256 = dct_ii(256)

Y256 = colxfm(colxfm(X, C256).T, C256).T
step256 = optimisation_DCT(np.std(X - quantise(X, 17)), Y256, C256, 1, 40, 100 )
Y256q = quantise(Y256, step256)
Z256q = colxfm(colxfm(Y256q.T, C256.T).T, C256.T)
Yr256 = regroup(Y256q, 256)
print(dctbpp(Yr, 256))
# print("compression ratio for 256x256: {}".format(compression_ratio_DCT(Z, Y256, 256, 17, step256)))
# print("number of bits in regrouped image: {}".format(dctbpp(regroup(Y256q, 256)/256, 256)))

fig, ax = plt.subplots()
plot_image(Z256q, ax=ax)
ax.set(title  = "256x256 DCT")

0.0


<IPython.core.display.Javascript object>

[Text(0.5, 1.0, '256x256 DCT')]

<div class="alert alert-block alert-danger">

Can you draw any conclusions about the best choice of transform size for the
Lighthouse image?  Try to postulate what features in other images might make your
conclusions different, and suggest why.</div>

In [83]:
# Your code here


transform_size = [2,4,8,16,32,64,128]
compression_ratios = []
number_of_bits = []
for t in transform_size:
    Ct = dct_ii(t)

    Yt = colxfm(colxfm(X, Ct).T, Ct).T
    stept = optimisation_DCT(np.std(X - quantise(X, 17)), Yt, Ct, 1, 40, 200 )
    Ytq = quantise(Yt, stept)
    Ztq = colxfm(colxfm(Ytq.T, Ct.T).T, Ct.T)
    number_of_bits.append(dctbpp(regroup(Ytq, t)/t, t))
    compression_ratios.append(compression_ratio_DCT(X, Yt, t, 17, stept))
print(compression_ratios)
for n in range(len(transform_size)):
    print("size: {}, compression ratio: {}, number of bits: {}".format(transform_size[n], compression_ratios[n], number_of_bits[n]))

[1.9417400511132894, 2.647441491067228, 2.946373050593443, 2.883369722681117, 4.143900690048731, 6.1631524618110465, 12.59600814786151]
size: 2, compression ratio: 1.9417400511132894, number of bits: 117481.75889347096
size: 4, compression ratio: 2.647441491067228, number of bits: 86165.84626643773
size: 8, compression ratio: 2.946373050593443, number of bits: 77423.6773828558
size: 16, compression ratio: 2.883369722681117, number of bits: 79115.43036755263
size: 32, compression ratio: 4.143900690048731, number of bits: 55049.3492922015
size: 64, compression ratio: 6.1631524618110465, number of bits: 37013.36903998226
size: 128, compression ratio: 12.59600814786151, number of bits: 18110.423067439537


In [84]:
fig, ax = plt.subplots()
transform_size = np.array(transform_size)
compression_ratios = np.array(compression_ratios)
ax.semilogx(transform_size, compression_ratios, "-o", base = 2)

<IPython.core.display.Javascript object>